In [1]:
pwd = '/home/zjy/project/MetaIM/data'

In [2]:
from torch_geometric.datasets import Planetoid

cora_dataset = Planetoid(root=pwd+'/cora', name='cora')
data = cora_dataset[0]
edge_index = data.edge_index